In [1]:
# In this cell, i will try to create the whole code
from bs4 import BeautifulSoup
import requests

# url = 'https://www.kathimerini.gr/tag/kyvernisi/page/1/'
url = input('Please give m the URL')
url = url+'page/1/'

#url = "https://www.kathimerini.gr/tag/akriveia/page/1/"

tname = input('print the tag name')
a = 'Kathimerini'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

# here, will be saved the final data:
final_tag_list = []
final_date_list = []

# the date which interests me (the final date)
searching_until = 20230731
flag = True

def new_url():
    global url
    number=[]
    n=0
    cnt_thesi=0
    for i in url:
        nr=i.isnumeric() 
        if nr==True:
            cnt_thesi=cnt_thesi+1
            number.append(i)
            if cnt_thesi==1:
                thesi=n
        n=n+1
    # now, im gona make the list number a number. And then, a string...
    number="".join(number)
    number=int(number)
    number=number+1
    number=str(number)
    url=url[:thesi]+number+"/"
    #print(url)
#The function that creates the next url:
# def new_url():
#     global url
#     result = []
#     for char in url:
#         if char.isdigit():
#             result.append(char)
#     # the number of the curr page:
#     digits = "".join(result)
#     digits = int(digits)
#     digits = 1+ digits
#     # the nr of the next page:
#     digits = str(digits)
#     url = url[:48]+digits+url[48+len(digits):]
#     print(url[:48]+digits+url[48+len(digits):])

# messanger is the messanger that informs for a broken url. Considering his message the date is not appended.
# It takes affect on the Catch comment
messanger=False
def core_function(page_url, searching_until):
    global final_tag_list
    global final_date_list
    global flag
    global messanger
    
    #global searching_until
    inside_tag_list = []
    def date_checker(article_url):
        global flag
        global messanger
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')
# Start of Patch. The notes are the old code. See notes.
        try:
            time=soup.find('time')
            if time==None:
                print('Atribute Error occured. \n The art url is ' , article_url)
                time = soup.find('span' , class_ = 'entry-meta-date')
                time = time.get_text()
                time = time[:10]
                time = time.split(".")
                time.reverse()
                time = "".join(time)
                time = int(time)
            else:
                time = time.get('datetime')
                time = time[:10].replace('-' , "")
                time = int(time)
            # try:
            #     time = soup.find('time')
            #     time = time.get('datetime')
            #     time = time[:10].replace('-' , "")
            #     time = int(time)
            # except AttributeError:
            #     print('atribute error occured.')
            #     print('the article url is ' , article_url)
            #     time = soup.find('span' , class_ = 'entry-meta-date')
            #     if time!=None:
            #         time = time.get_text()
            #         time = time[:10]
            #         time = time.split(".")
            #         time.reverse()
            #         time = "".join(time)
            #         time = int(time)
            #     else:
            if time < searching_until:
                # edw prepei n valw to stopping point mou :
                print('the date is reached. Stop the loop!!!')
                flag = False
            #return
            
            else:
                #edw einai h fasi pou prepei na sunexisei o kwdikas na trexei
                #print('sunexizw kanonika')
                final_date_list.append(time)
        except:
            print('broken article. Its url is ' , article_url)
# End of Patch  

    def tag_saver(article_url):
        global messanger
        messanger=False
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        mother = soup.find("div" , class_ = "tags-wrapper")
        if mother==None:
            print('Attention! vrika xalasmeno arthro me url: ' , article_url)
            messanger=True
        else:
            try:
                find_a = mother.find_all('a')

                for a in find_a:
                    tag = a.get_text()
                    inside_tag_list.append(tag)
            except:
                print('PLease check the article url. \n its Propably broken. ' , article_url)


    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
   
    # it finds the mother of the html tags that interests me: the date and the outside tag:
    media_content = soup.find_all('div' , class_ = "media-content")
    # If the length of the 'media_content is 0 , it means that there are no articles. So, the loop must terminate immidiatly 
    if len(media_content) == 0 :
        print('len(media_content) == 0')
        flag = False

    for i in media_content:
        article_url = i.find('a')
        article_url = article_url.get('href')
        # print(article_url)
        # i've got the article's url, so, i now call the date checker.
        date_checker(article_url)
        if flag==True:
            # the date_checker desides if the programm (loop) will continue or not. The below code is the case where the date_checker allows the conitnuety:
            tag_saver(article_url)
            outer_tag = i.find('p' , class_ = "subtitle is-7 mb-3")
            outer_tag = outer_tag.get_text()
# Catch
            if messanger== False:
                inside_tag_list.append(outer_tag)
                final_tag_list.append(inside_tag_list)
            else:
                print('the messanger is True. The article is broken and neither tag nor date is appended. See it at the end , comparing their lengths')
# End of Catch
            inside_tag_list = []
        else:
            break
    # if the length of the articles is less than 15, it means that there is no next page. So, the while loop must break!!
    if len(media_content) < 15:
        print('len(media_content) < 15')
        flag = False


flag = True
while flag:
    core_function(url,searching_until)
    if flag==False:
        print('the while loop has stopped immediatly')
        break
    #print('next while loop')
    new_url()
    

print(len(final_date_list))
print(len(final_tag_list))
print('the last date collected is ' , final_date_list[-1])
print(final_tag_list)


import pandas as pd
# df_costliness = pd.DataFrame(tag_list,date_list)
# df_costliness.to_csv('costliness.csv')

#print(tag_list)
#print(date_list)

new_tag_list = []
for i in range(len(final_tag_list)):
    for j in range(len(final_tag_list[i])):
        #if tag_list[i][j] not in new_tag_list:
       new_tag_list.append(final_tag_list[i][j])
#print(new_tag_list)

res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

resultList = new_list = list(map(list, res.items()))
# printing the resultant list of a dictionary key-values
print(resultList)

df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)


# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=final_date_list
ttag_list = final_tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)

n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    for tag in i:
        inside=[]
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)


print(len(final_tag_list))
print(len(final_date_list))
print(final_tag_list)
print('the last date collected is ' , final_date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563153971/anazopyrosi-plithoristikon-pieseon-ton-ioylio-stin-ellada/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563144647/meiosi-timon-se-galaktokomika-kai-eidi-atomikis-ygieinis/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563142838/oi-diakopes-eginan-mini-apodraseis/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563140786/epitropi-antagonismoy-sti-machi-kata-tis-akriveias-choris-prosopiko/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563131678/i-aktinografia-tis-akriveias-ti-apokalyptei-ekthesi-tis-tte-gia-tis-times-sta-soyper-market/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563118685/akriveia-diakopes-to-2024-me-misthoys-toy-2012/
Atribute Error occured. 
 The art url is  https://www.kathimerini.gr/economy/563068180/statheres-oi-times-sta-peris

In [ ]:
url = 'https://www.kathimerini.gr/politics/563022658/episkepsi-mitsotaki-stin-toyrkia-lepto-pros-lepto-oi-exelixeis/'


In [5]:
df_costliness = pd.DataFrame(final_date_list)
df_costliness.to_csv('Kath_Kur_MHTs_Datelist.csv', header=False)
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv('Kath_Kur_Mhts_Taglist.csv', header=False)

In [14]:
# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=final_date_list
ttag_list = final_tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)
print(len(ddate_list))
print(len(ttag_list))
n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    print('the n is ' , n)
    for tag in i:
        inside=[]
        inside.append(tag)
        try:
            inside.append(ddate_list[n-1])
            date_con_list.append(inside)
        except:
            inside.append("")
            date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

[20240802, 20240802, 20240801, 20240801, 20240731, 20240731, 20240730, 20240730, 20240729, 20240729, 20240729, 20240729, 20240728, 20240727, 20240727, 20240726, 20240725, 20240724, 20240724, 20240724, 20240724, 20240723, 20240722, 20240722, 20240721, 20240721, 20240721, 20240721, 20240721, 20240720, 20240720, 20240720, 20240719, 20240718, 20240718, 20240718, 20240718, 20240718, 20240718, 20240717, 20240717, 20240717, 20240717, 20240716, 20240716, 20240716, 20240715, 20240715, 20240715, 20240715, 20240715, 20240715, 20240714, 20240714, 20240714, 20240713, 20240713, 20240713, 20240712, 20240712, 20240712, 20240711, 20240711, 20240711, 20240710, 20240710, 20240710, 20240710, 20240710, 20240709, 20240709, 20240709, 20240709, 20240709, 20240708, 20240708, 20240708, 20240708, 20240707, 20240707, 20240707, 20240706, 20240706, 20240705, 20240705, 20240705, 20240705, 20240704, 20240704, 20240703, 20240703, 20240703, 20240703, 20240702, 20240702, 20240702, 20240702, 20240701, 20240701, 20240701,

In [11]:
# In this cell, i will try to create the whole code
from bs4 import BeautifulSoup
import requests

url = 'https://www.kathimerini.gr/tag/kyvernisi/page/34/'
# url = input('Please give m the URL')
# url = url+'page/1/'

#url = "https://www.kathimerini.gr/tag/akriveia/page/1/"

tname = input('print the tag name')
a = 'Kathimerini'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

# here, will be saved the final data:
final_tag_list = []
final_date_list = []

# the date which interests me (the final date)
searching_until = 20230731
flag = True

def new_url():
    global url
    result = []
    for char in url:
        if char.isdigit():
            result.append(char)
    # the number of the curr page:
    digits = "".join(result)
    digits = int(digits)
    digits = 1+ digits
    # the nr of the next page:
    digits = str(digits)
    url = url[:46]+digits+url[46+len(digits):]
    #print(url[:48]+digits+url[48+len(digits):])
    #print(url)
#The function that creates the next url:
# def new_url():
#     global url
#     result = []
#     for char in url:
#         if char.isdigit():
#             result.append(char)
#     # the number of the curr page:
#     digits = "".join(result)
#     digits = int(digits)
#     digits = 1+ digits
#     # the nr of the next page:
#     digits = str(digits)
#     url = url[:48]+digits+url[48+len(digits):]
#     print(url[:48]+digits+url[48+len(digits):])


def core_function(page_url, searching_until):
    global final_tag_list
    global final_date_list
    global flag
    #global searching_until
    inside_tag_list = []
    def date_checker(article_url):
        global flag
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            time = soup.find('time')
            time = time.get('datetime')
            time = time[:10].replace('-' , "")
            time = int(time)
        except AttributeError:
            time = soup.find('span' , class_ = 'entry-meta-date')
            time = time.get_text()
            time = time[:10]
            time = time.split(".")
            time.reverse()
            time = "".join(time)
            time = int(time)
        if time < searching_until:
            # edw prepei n valw to stopping point mou :
            print('the date is reached. Stop the loop!!!')
            flag = False
            #return
            
        else:
            #edw einai h fasi pou prepei na sunexisei o kwdikas na trexei
            #print('sunexizw kanonika')
            final_date_list.append(time)
    

    def tag_saver(article_url):
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        mother = soup.find("div" , class_ = "tags-wrapper")
        print('the mother is ' , mother)
        find_a = mother.find_all('a')
        #print('the find a is ' ,find_a)

        for a in find_a:
            tag = a.get_text()
            inside_tag_list.append(tag)


    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, 'html.parser')
   
    # it finds the mother of the html tags that interests me: the date and the outside tag:
    media_content = soup.find_all('div' , class_ = "media-content")
    # If the length of the 'media_content is 0 , it means that there are no articles. So, the loop must terminate immidiatly 
    if len(media_content) == 0 :
        print('len(media_content) == 0')
        flag = False

    for i in media_content:
        article_url = i.find('a')
        article_url = article_url.get('href')
        # print(article_url)
        # i've got the article's url, so, i now call the date checker.
        date_checker(article_url)
        if flag==True:
            # the date_checker desides if the programm (loop) will continue or not. The below code is the case where the date_checker allows the conitnuety:
            tag_saver(article_url)
            outer_tag = i.find('p' , class_ = "subtitle is-7 mb-3")
            outer_tag = outer_tag.get_text()
            inside_tag_list.append(outer_tag)
            final_tag_list.append(inside_tag_list)
            inside_tag_list = []
        else:
            break
    # if the length of the articles is less than 15, it means that there is no next page. So, the while loop must break!!
    if len(media_content) < 15:
        print('len(media_content) < 15')
        flag = False


flag = True
while flag:
    core_function(url,searching_until)
    if flag==False:
        print('the while loop has stopped immediatly')
        break
    #print('next while loop')
    new_url()
    

print(len(final_date_list))
print(len(final_tag_list))
print('the last date collected is ' , final_date_list[-1])
print(final_tag_list)


import pandas as pd
# df_costliness = pd.DataFrame(tag_list,date_list)
# df_costliness.to_csv('costliness.csv')

#print(tag_list)
#print(date_list)

new_tag_list = []
for i in range(len(final_tag_list)):
    for j in range(len(final_tag_list[i])):
        #if tag_list[i][j] not in new_tag_list:
       new_tag_list.append(final_tag_list[i][j])
#print(new_tag_list)

res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

resultList = new_list = list(map(list, res.items()))
# printing the resultant list of a dictionary key-values
print(resultList)

df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)


# constructs the date, ready for tableau, with the form day, month, year with dots. Saves it in the var 'ndlist'
ddate_list=final_date_list
ttag_list = final_tag_list

ndlist = []
for i in ddate_list:
    i=str(i)
    y=i[:4]
    m=i[4:6]
    d=i[6:]
    # here, i am gona crate the date with dots and the system: day,month, year:
    ddate = d+'.'+m+'.'+y
    ndlist.append(ddate)
print(ddate_list)
print(ndlist)

n=0
date_con_list = []
for i in ttag_list:
    n=n+1
    for tag in i:
        inside=[]
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)


print(len(final_tag_list))
print(len(final_date_list))
print(final_tag_list)
print('the last date collected is ' , final_date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


the mother is  <div class="tags-wrapper">
<div>
<span class="tags-links"><a href="https://www.kathimerini.gr/tag/voridis-makis/" rel="tag">Βορίδης Μάκης</a> <a href="https://www.kathimerini.gr/tag/ekklisia/" rel="tag">Εκκλησία</a> <a href="https://www.kathimerini.gr/tag/kyvernisi/" rel="tag">Κυβέρνηση</a> <a href="https://www.kathimerini.gr/tag/mitsotakis-kyriakos/" rel="tag">Κυριάκος Μητσοτάκης</a> <a href="https://www.kathimerini.gr/tag/maximoy/" rel="tag">Μαξίμου</a> <a href="https://www.kathimerini.gr/tag/nea-dimokratia/" rel="tag">Νέα Δημοκρατία</a> <a href="https://www.kathimerini.gr/tag/nomoschedio/" rel="tag">Νομοσχέδιο</a> <a href="https://www.kathimerini.gr/tag/omofyla-zeygaria/" rel="tag">Ομόφυλα ζευγάρια</a> <a href="https://www.kathimerini.gr/tag/ypoyrgiko-symvoylio/" rel="tag">Υπουργικό συμβούλιο</a></span> </div>
</div>
the mother is  <div class="tags-wrapper">
<div>
<span class="tags-links"><a href="https://www.kathimerini.gr/tag/voyli/" rel="tag">ΒΟΥΛΗ</a> <a href="htt

AttributeError: 'NoneType' object has no attribute 'find_all'